In [ ]:
class Movalyst:
    def __init__(self):
        return self
    # get all urls for movie list from year x to year y
    def get_movie_url(x=1980,y=2018):
        import requests 
        import datetime
        from bs4 import BeautifulSoup
    
        if x > y :
            raise ValueError('Start time should be earlier.')
        if x < 1980 or y> int(str(datetime.datetime.now())[:4]):
            raise ValueError('Please input time during 1980 till now.')
    
        url = 'https://www.boxofficemojo.com/yearly/'
        url_list = []
    
        response = requests.get(url)
        results = BeautifulSoup(response.content, 'lxml')
        results = results.find_all('a')
    
        for result in results:
            if 'yr=' in result.get('href') and int(result.get_text())>=x and int(result.get_text())<=y:
                url_list.append(url+result.get('href'))
        return url_list

    def get_movie(url = None, year = None):
        import requests 
        from bs4 import BeautifulSoup
        if url == None:
            url = 'https://www.boxofficemojo.com/yearly/chart/?yr='+str(year)+'&p=.htm'
        response = requests.get(url)
        results = BeautifulSoup(response.content, 'lxml')
        results = results.find_all('a')        
    
        url_list=[]
        for result in results:
            if result.get('href') != None:
                if 'id='in result.get('href') and 'grinch' not in result.get('href'):
                    url_list.append('https://www.boxofficemojo.com'+result.get('href'))
        return url_list
    
    def get_movie_name(url = None, year = None):
        import requests 
        from bs4 import BeautifulSoup
        if url == None:
        url = 'https://www.boxofficemojo.com/yearly/chart/?yr='+str(year)+'&p=.htm'
        response = requests.get(url)
        results = BeautifulSoup(response.content, 'lxml')
        results = results.find_all('a')        
    
        moviename_list=[]
        for result in results:
            if result.get('href') != None:
                if 'id='in result.get('href') and 'grinch' not in result.get('href'):
                    moviename_list.append(result.get_text())
        return moviename_list
    
    def find_all(x,y):
        import requests 
        from bs4 import BeautifulSoup
        list_url = get_movie_url(x,y)
        dict_movie = {}
        for url in list_url:
            list_0 = list_movie.copy()
            list_1 = dict(map(lambda x,y:[x,y],get_movie_name(url),get_movie(url)))
            dict_movie = {**list_0, **list_1}
            return dict_movie
    
    def data_movie(year):
        
        import requests 
        from bs4 import BeautifulSoup
        url = 'https://www.boxofficemojo.com/yearly/chart/?yr='+str(year)+'&p=.htm'
        response = requests.get(url)
        results = BeautifulSoup(response.content, 'lxml')
        results = results.find_all('font',size="2")
        return results
    
    # get a movie's daily gross
    def gross_movie(name,year):      
        import pandas as pd
        import requests 
        from bs4 import BeautifulSoup
        movie_dict = find_all(year,year)
        try:
            movie_url = find_all(2017,2018)['Black Panther']
            id_position = movie_url.find('id=')
            name_id = movie_url[id_position:]
            url = 'https://www.boxofficemojo.com/movies/?page=daily&view=chart&'+name_id
            response = requests.get(url)
            results = BeautifulSoup(response.content, 'lxml')
            tables = results.select('table')
            df_list = []
            for table in tables:
                df_list.append(pd.concat(pd.read_html(table.prettify())))
            movie_gross = df_list[7]
            movie_gross.columns = ['Day','Date','Rank','Gross','%daily-change','%weekly-change','Theaters','Average','Gross-to-Date','Days']
            movie_gross = movie_gross[(movie_gross['Date'] != 'Date') & (movie_gross['Date'].isnull()-1)]
            movie_gross.set_index('Date',inplace=True)
            movie_gross[['Days']] = movie_gross[['Days']].astype(int)
            return movie_gross        
        except:
            ValueError(f'{name} is not in the TOP100 of year {year}')

In [ ]:
# Then ,we will scrapy movie features from the website
